<div style="text-align: right"><strong>Capstone #3:</strong> <span style="color:darkred">Supervised Learning</span> </div>

<a id="top"></a>

#### <span style="color:darkred">__Part 1: Data Exploration__ https://github.com/kimrharper/thinkful/blob/master/unit3/unit3-capstone-exploration.ipynb </span><br><br><span style="color:darkred">__Part 2: Models__ https://github.com/kimrharper/thinkful/blob/master/unit3/unit3-capstone-models.ipynb </span>

----

# <span style="color:darkred">Part 2: </span><span style="color:darkblue">L1 Prediction from ELL Writing Samples</span>

__Author:__ Ryan Harper 

----

<a href='#ov'>Overview</a><br>
<a href='#exp'>Experiment</a><br>
<a href='#sec1'>1. Models:</a><br>
><a href='#seca'>A. LR - Ordinary Least Squares</a><br>
<a href='#secb'>B. LR - Logistic Regression</a> <a href='#secb1'> (Lasso)</a> <a href='#secb2'> (Ridge)</a><br>
<a href='#secc'>C. NN - K Nearest Neighbors</a><br>
<a href='#secd'>D. NN - Naive Bayes</a><br>
<a href='#sece'>E. NN - Decision Tree</a><br>
<a href='#secf'>F. Ensemble - Random Forest</a><br>

<a href='#sec2'>2. Model Comparison</a><br>

<a id="ov"></a>

# <span style="color:darkblue">Overview</span>  <a href='#top'>(top)</a>

__Data Source:__
> http://lang-8.com/ [scraped with Beautiful Soup]

![alt text](../data/language/lang8.png "Title")

__Summary:__
> In my previous profession, I have been teaching English to a diverse range of students of all ages, language background, and country origin. During my professional development, I started to observe that different students with different L1s (1st Language) tended to display different patterns of communication that appeared to have some connection to either education in their country of origin or a connection to the linguistic structure of their first language. Different ELL (English Language Learners) needed to focus on different aspects of the English language depending on their background. The purpose of this project is to use a large number of blog posts from a language practicing website and explore whether or not the L1 has any significant impact on the blog writing style of the English learner.<br><br>Part 1: Explore the data to find any noteworthy trends in linguistic structure: <ol><li> vocabulary (word freq, collocations, and cognates) <li>syntax (sentence structure)<li>grammar (i.e. grammar complexity of sentences) <li>errors (types of errors) <li> parts of speech (NLTK Abbreviations: https://pythonprogramming.net/natural-language-toolkit-nltk-part-speech-tagging/)</ol><br>Part 2: Use linguistic trends to determine whether or not a learner's first language can be predicted.

__Variables:__
>__id:__ _User ID_<br>
__time:__ _Time the blog post was scraped (in order of user posted time)_ <br>
__title:__ _Title of the blog post_<br>
__content:__ _The blog post_<br>
__language:__ _User's self-reported first language_

<a id="exp"></a>

# <span style="color:darkblue">Experiment</span> <a href='#top'>(top)</a>

__Hypothesis:__ 
> L1 (first language) experience and academic environment influences ELLs' (English Language Learners') writing style. The L1 of ELLs can be predicted by looking at English blog posts and identifying patterns unique to their L1.

__Observations:__
><li> --<li>--<li>--

__Method:__
> Using multiple different models. The aim of this project is to explore how different models can handle the data (target and features) and to see what information can be gained from using multiple different models. Ultimately, the goal is to determine which models are appropriate for a binary (discrete) target with features that are both qualitative (discrete) and quantitative (ranked/continuous).

<a id="sec1"></a>

# <span style="color:darkblue">1. Models:</span>  <a href='#top'>(top)</a>

In [1]:
# iPython/Jupyter Notebook
import time
from pprint import pprint
import warnings
from IPython.display import Image

import time

# Data processing
import pandas as pd
import plotly as plo
import seaborn as sns
from scipy import stats
from collections import Counter
import numpy as np
import itertools

# NLP
from nltk.corpus import stopwords as sw
from nltk.util import ngrams
from nltk.corpus import brown
import nltk
import re
from nltk.tokenize import RegexpTokenizer
import difflib

# Stats
from sklearn.metrics import classification_report, roc_curve,roc_auc_score,accuracy_score
from sklearn import metrics

# Preparing Models
from sklearn.model_selection import train_test_split

# Models
from sklearn import linear_model
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.naive_bayes import BernoulliNB,MultinomialNB,GaussianNB

# Ensemble
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

#Visualization
from IPython.display import Image
import pydotplus
import graphviz

# import altair as alt

In [2]:
features = pd.read_csv('blogfeatures.csv').sample(frac=1.0)
del features['Unnamed: 0']
del features['id']
lang = list(features.language.unique())

In [3]:
y = features['language'].values.reshape(-1, 1).ravel()
X = features[features.columns[~features.columns.str.contains('language')]]
X.head()

print(np.shape(y))
print(np.shape(X))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=32)  

(13087,)
(13087, 474)


<a id="seca"></a>

__Create Function for Comparing Models__

In [4]:
cols = ['name','time','total','precision','recall','f1']

model_set = pd.DataFrame(columns=cols)
models_stored = []
pattern = "%.2f"

In [5]:
def run_model(model,name):
    global model_set
    m = model
    m.fit(X_train, y_train)
    start = time.time()

    total_score = m.score(X_test,y_test)
    pscore = [pattern % i for i in list(metrics.precision_score(y_test, m.predict(X_test),labels=lang,average=None))]
    rscore = [pattern % i for i in list(metrics.recall_score(y_test, m.predict(X_test),labels=lang,average=None))]
    fscore = [pattern % i for i in list(metrics.f1_score(y_test, m.predict(X_test),labels=lang,average=None))]
    end = time.time()
    t= pattern % (end - start)

    r = dict(zip(cols,[name,t,total_score,pscore,rscore,fscore]))
    print('Total Score is: {}\n'.format(total_score))
    print(classification_report(y_test, m.predict(X_test)))
    
    model_set = model_set.append(r,ignore_index=True)
    return r,m

### <span style="color:gray">A. LR - Ordinary Least Squares _(not used)_</span>  <a href='#top'>(top)</a>

> Target is discrete so this model may not be appropriate <br>Many features are binary so model may not be appropriate

<a id="secb"></a>

### <span style="color:darkred">B. LR - Logistic Regression</span>  <a href='#top'>(top)</a>

> Target is binary so logistic regression will operate on probabilities

In [6]:
%%time
lreg_data,lreg = run_model(linear_model.LogisticRegression(),'Logistic Regression')

Total Score is: 0.7673116089613035

                     precision    recall  f1-score   support

            English       0.00      0.00      0.00         4
           Japanese       0.79      0.93      0.86      1355
             Korean       0.29      0.03      0.06       129
Traditional Chinese       0.68      0.50      0.58       476

        avg / total       0.73      0.77      0.73      1964

CPU times: user 3.39 s, sys: 63.7 ms, total: 3.45 s
Wall time: 3.49 s


/Users/ryan/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples.

/Users/ryan/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 in labels with no predicted samples.

/Users/ryan/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.



In [19]:
pd.crosstab(y_test,lreg.predict(X_test))

col_0,Japanese,Korean,Traditional Chinese
row_0,,,
English,2,0,2
Japanese,1264,6,85
Korean,102,4,23
Traditional Chinese,233,4,239


<a id="secb1"></a>

### <span style="color:gray">C. Lasso _(not used)_</span>  <a href='#top'>(top)</a>

<a id="secb2"></a>

### <span style="color:gray">D. Ridge _(not used)_</span>  <a href='#top'>(top)</a>

_Lasso and Ridge are not good predictors so should I just be using them for parameter manipulation?_

<a id="sece"></a>

### <span style="color:darkred">E. K Nearest Neighbors</span>  <a href='#top'>(top)</a>

> Can handle discrete values for target <br>Quantitative values are limited (not continuous) and might be problematic for nearest neighbors

In [8]:
%%time
neighbors_data,neighbors = run_model(KNeighborsClassifier(n_neighbors=5),'K Nearest Neighbor')

Total Score is: 0.6863543788187373

                     precision    recall  f1-score   support

            English       0.00      0.00      0.00         4
           Japanese       0.72      0.92      0.81      1355
             Korean       0.00      0.00      0.00       129
Traditional Chinese       0.48      0.20      0.29       476

        avg / total       0.61      0.69      0.63      1964

CPU times: user 3.12 s, sys: 29.7 ms, total: 3.15 s
Wall time: 3.16 s


<a id="secf"></a>

### <span style="color:darkred">F. Naive Bayes - Bernoulli</span>  <a href='#top'>(top)</a>

> Should be best for boolean classification but not for multiple discrete values (multinomial should be better)

In [9]:
%%time
bnb_data,bnb = run_model(BernoulliNB(),'Naive Bayes - Bernoulli')

Total Score is: 0.6064154786150713

                     precision    recall  f1-score   support

            English       0.00      0.00      0.00         4
           Japanese       0.75      0.74      0.74      1355
             Korean       0.09      0.14      0.11       129
Traditional Chinese       0.46      0.35      0.40       476

        avg / total       0.63      0.61      0.62      1964

CPU times: user 213 ms, sys: 71.2 ms, total: 284 ms
Wall time: 289 ms


<a id="secg"></a>

### <span style="color:darkred">G. Decision Tree</span>  <a href='#top'>(top)</a>

> Visualizes most important features by hierarchy <br>Longer processing time

In [10]:
%%time
dt_data,dt = run_model(tree.DecisionTreeClassifier(criterion='entropy',max_depth=6),'Decision Tree')

Total Score is: 0.7209775967413442

                     precision    recall  f1-score   support

            English       0.00      0.00      0.00         4
           Japanese       0.75      0.93      0.83      1355
             Korean       0.00      0.00      0.00       129
Traditional Chinese       0.56      0.32      0.41       476

        avg / total       0.65      0.72      0.67      1964

CPU times: user 295 ms, sys: 23.2 ms, total: 318 ms
Wall time: 316 ms


/Users/ryan/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples.

/Users/ryan/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 in labels with no predicted samples.

/Users/ryan/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.



In [11]:
# Render tree.
dot_data = tree.export_graphviz(
    dt, out_file=None,
    feature_names=X.columns,
    class_names=lang,
    filled=True
)

graph = pydotplus.graph_from_dot_data(dot_data)
Image(graph.create_png())

graph.write_png('decision_tree.png')

True

In [12]:
dimportance = list(zip(X.columns,dt.feature_importances_))
dimportance = dict(dimportance)
a1_sorted_keys = sorted(dimportance, key=dimportance.get, reverse=True)
p = []
for r in a1_sorted_keys:
    if dimportance[r] != 0:
        p.append(r)
#         print(r, dimportance[r])
        
print(p)

['sc', 'pos2_NN-MD', 'wc', 'pos2_DT-JJ', 'pos2_PRP-VBD', 'pos2_NN-PRP', 'pos2_PRP-RB', 'pos2_NN-TO', 'pos2_MD-VB', 'sent_pol', 'pos2_NN-RB', 'pos2_WRB-NN', 'pos2_RB-JJ', 'sent_subj', 'pos2_PRP$-NN', 'pos2_NNP-JJ', 'pos2_IN-NNP', 'pos2_VBP-VBG', 'pos2_WRB-MD', 'pos2_VBP-RB', 'pos2_VBZ-DT', 'pos2_RB-NN', 'pos2_VB-IN', 'pos2_VBZ-RB', 'pos2_RB-VB', 'pos2_DT-PRP$', 'pos2_VB-PRP', 'pos2_IN-NN', 'pos2_WDT-VBD', 'pos2_VBZ-PRP', 'pos2_NN-JJ', 'pos2_IN-VBG', 'pos2_CC-RB', 'pos2_EX-VBD', 'pos2_NNS-CC']


_Good visualization of important features and presentation of entropy weighting_

<a id="sech"></a>

### <span style="color:darkred">H. Random Forest</span>  <a href='#top'>(top)</a>

> Runs decision tree multiple times for best output <br>Longest processing time

In [13]:
%%time
rf_data,rf = run_model(ensemble.RandomForestClassifier(n_estimators=20),'Random Forest')

Total Score is: 0.7337067209775967

                     precision    recall  f1-score   support

            English       0.00      0.00      0.00         4
           Japanese       0.74      0.96      0.84      1355
             Korean       0.00      0.00      0.00       129
Traditional Chinese       0.67      0.30      0.41       476

        avg / total       0.67      0.73      0.68      1964

CPU times: user 836 ms, sys: 50.3 ms, total: 887 ms
Wall time: 900 ms


/Users/ryan/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples.

/Users/ryan/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 in labels with no predicted samples.

/Users/ryan/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.



In [14]:
cvs = cross_val_score(rf, X_test, y_test, cv=5)
print(cvs)
scoreH = cvs.sum()/len(cvs)

/Users/ryan/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning:

The least populated class in y has only 4 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.



[0.70558376 0.71246819 0.72264631 0.71501272 0.71611253]


In [15]:
rf.score(X_train,y_train)
print(scoreH)

0.7143647029555182


<a id="sec2"></a>

# <span style="color:darkblue">2. Model Comparison</span>  <a href='#top'>(top)</a>

In [16]:
model_set.columns = ['name','time','total','prec: | JA | CH | KO | EN |','rec: | JA | CH | KO | EN |','f1: | JA | CH | KO | EN |']
model_set

,name,time,total,prec: | JA | CH | KO | EN |,rec: | JA | CH | KO | EN |,f1: | JA | CH | KO | EN |
0,Logistic Regression,0.03,0.767312,"[0.79, 0.68, 0.29, 0.00]","[0.93, 0.50, 0.03, 0.00]","[0.86, 0.58, 0.06, 0.00]"
1,K Nearest Neighbor,2.35,0.686354,"[0.72, 0.48, 0.00, 0.00]","[0.92, 0.20, 0.00, 0.00]","[0.81, 0.29, 0.00, 0.00]"
2,Naive Bayes - Bernoulli,0.08,0.606415,"[0.75, 0.46, 0.09, 0.00]","[0.74, 0.35, 0.14, 0.00]","[0.74, 0.40, 0.11, 0.00]"
3,Decision Tree,0.04,0.720978,"[0.75, 0.56, 0.00, 0.00]","[0.93, 0.32, 0.00, 0.00]","[0.83, 0.41, 0.00, 0.00]"
4,Random Forest,0.07,0.733707,"[0.74, 0.67, 0.00, 0.00]","[0.96, 0.30, 0.00, 0.00]","[0.84, 0.41, 0.00, 0.00]"


-----